In [1]:
import cv2
import mediapipe as mp
import numpy as np
import time, os
from tqdm import tqdm
import gc

np.seterr(divide='ignore', invalid='ignore')

objc[2559]: Class CaptureDelegate is implemented in both /Users/user/miniforge3/envs/tensorflow/lib/python3.9/site-packages/cv2/cv2.abi3.so (0x1465e2840) and /Users/user/miniforge3/envs/tensorflow/lib/python3.9/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x135de4860). One of the two will be used. Which one is undefined.
objc[2559]: Class CVWindow is implemented in both /Users/user/miniforge3/envs/tensorflow/lib/python3.9/site-packages/cv2/cv2.abi3.so (0x1465e2890) and /Users/user/miniforge3/envs/tensorflow/lib/python3.9/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x1154aca68). One of the two will be used. Which one is undefined.
objc[2559]: Class CVView is implemented in both /Users/user/miniforge3/envs/tensorflow/lib/python3.9/site-packages/cv2/cv2.abi3.so (0x1465e28b8) and /Users/user/miniforge3/envs/tensorflow/lib/python3.9/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x1154aca90). One of the two will be used. Which one is u

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
#import pandas as pd
trainpath = "/Users/user/PycharmProjects/DL_hands/open/train/"
#df = pd.read_csv("/Users/user/PycharmProjects/DL_hands/open/train.csv")

In [3]:
def fileload(num):
    path = trainpath
    filename = ""
    if num<10:
        filename = "TRAIN_"+"00"+str(num)+".mp4"
    elif num<100:
        filename = "TRAIN_"+"0"+ str(num)+ ".mp4"
    else:
        filename = "TRAIN_"+ str(num)+ ".mp4"

    cap = cv2.VideoCapture(path + filename)
    #width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    #height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    #fps = cap.get(cv2.CAP_PROP_FPS)
    
    return cap

In [4]:
mpHands = mp.solutions.hands
hands = mpHands.Hands(max_num_hands=1,
                      min_detection_confidence=0.5,
                      min_tracking_confidence=0.5)
mpDraw = mp.solutions.drawing_utils

In [5]:
cd /Users/user/PycharmProjects/DL_hands/open/train

/Users/user/PycharmProjects/DL_hands/open/train


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [100]:
def play(cap):
    data = np.array([])

    while cap.isOpened():
        ret, img = cap.read()

        if ret:
            imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            results = hands.process(imgRGB)
            h, w, c = img.shape

            ###### Data 
            if results.multi_hand_landmarks:
                for res in results.multi_hand_landmarks:
                    mpDraw.draw_landmarks(img, res, mpHands.HAND_CONNECTIONS)
                    joint = np.zeros((21, 3))
                    for id, lm in enumerate(res.landmark):
                        joint[id] = [lm.x, lm.y, lm.z]

                    # Compute Angles (1) (2) (3) Fingers
                    v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19], :3] # Parent joint
                    v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], :3] # Child joint
                    v = v2 - v1

                    v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

                    angle = np.arccos(np.einsum('nt,nt->n',
                        v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                        v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) # (15,0)
                    
                    # rad 2 deg
                    angle = np.degrees(angle)
                    angle = np.array(angle, dtype=np.int32)

                    tmp = np.array([angle])
                    tmp = tmp.flatten()
                    data = np.append(data, tmp)
                    
        else:
            cap.release()
            break

        cv2.imshow("Image", img)
        cv2.waitKey(1)
        cv2.destroyAllWindows()
    return np.array(data, dtype=np.int32)

In [7]:
#filetotal = len(os.listdir(trainpath))
filetotal = 610

In [8]:
def npsave(n):
    tmp = play(fileload(n))
    if (len(tmp) > 0):
        if n<10:
            with open('/Users/user/PycharmProjects/DL_hands/open/data/fingers_info00'+str(n), "wb") as f:
                np.save(f, tmp)
        elif n<100:
            with open('/Users/user/PycharmProjects/DL_hands/open/data/fingers_info0'+str(n), "wb") as f:
                np.save(f, tmp)
        else:
            with open('/Users/user/PycharmProjects/DL_hands/open/data/fingers_info'+str(n), "wb") as f:
                np.save(f, tmp)
        return None
    else:
        return n

In [9]:
failed_clips = []
for i in tqdm(range(filetotal)):
    num = npsave(i)
    
    if num != None:
        failed_clips.append(num)
    
    

100%|█████████████████████████████████████████| 610/610 [03:59<00:00,  2.54it/s]


In [29]:
np.array(failed_clips)
failed_clips = [  8,  74,  82, 126, 149, 211, 213, 278, 284, 312, 386, 457, 564]

In [11]:
"""
for i in failed_clips:
    if npsave(i) == None:
        failed_clips.remove(i)
        print(i, "success")
    else:
        continue
"""

'\nfor i in failed_clips:\n    if npsave(i) == None:\n        failed_clips.remove(i)\n        print(i, "success")\n    else:\n        continue\n'

In [12]:
np.save('fails.npy', failed_clips)

In [115]:
def saturate_contrast(p, num):
    # num eg) 1.xx 
    pic = p.copy()
    pic = pic.astype('int64')
    pic = np.clip(pic*num, 0, 255)
    pic = pic.astype('uint8')
    return pic

In [116]:
def play2(cap):
    data = np.array([])

    while cap.isOpened():
        ret, img = cap.read()

        if ret:
            imgRGB = saturate_contrast(img, 2.5)
            imgRGB = cv2.cvtColor(imgRGB, cv2.COLOR_BGR2RGB)
            results = hands.process(imgRGB)
            h, w, c = img.shape

            ###### Data 
            if results.multi_hand_landmarks:
                for res in results.multi_hand_landmarks:
                    mpDraw.draw_landmarks(img, res, mpHands.HAND_CONNECTIONS)
                    joint = np.zeros((21, 3))
                    for id, lm in enumerate(res.landmark):
                        joint[id] = [lm.x, lm.y, lm.z]

                    # Compute Angles (1) (2) (3) Fingers
                    v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19], :3] # Parent joint
                    v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], :3] # Child joint
                    v = v2 - v1

                    v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

                    angle = np.arccos(np.einsum('nt,nt->n',
                        v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                        v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) # (15,0)
                    
                    # rad 2 deg
                    angle = np.degrees(angle)
                    angle = np.array(angle, dtype=np.int32)

                    tmp = np.array([angle])
                    tmp = tmp.flatten()
                    data = np.append(data, tmp)
                    
        else:
            cap.release()
            break
        img2 = cv2.cvtColor(imgRGB, cv2.COLOR_RGB2BGR)
        cv2.imshow("Image", img2)
        cv2.waitKey(1)
        cv2.destroyAllWindows()
    return np.array(data, dtype=np.int32)

In [117]:
def npsave2(n):
    tmp = play2(fileload(n))
    if (len(tmp) > 0):
        if n<10:
            with open('/Users/user/PycharmProjects/DL_hands/open/data2/fingers_info00'+str(n), "wb") as f:
                np.save(f, tmp)
        elif n<100:
            with open('/Users/user/PycharmProjects/DL_hands/open/data2/fingers_info0'+str(n), "wb") as f:
                np.save(f, tmp)
        else:
            with open('/Users/user/PycharmProjects/DL_hands/open/data2/fingers_info'+str(n), "wb") as f:
                np.save(f, tmp)
        return None
    else:
        return n

In [98]:
failed_clips2 = []
for i in tqdm(failed_clips):
    num = npsave2(i)
    
    if num != None:
        failed_clips2.append(num)
    

100%|███████████████████████████████████████████| 13/13 [00:31<00:00,  2.45s/it]


In [35]:
np.array(failed_clips)

array([  8,  74,  82, 126, 149, 211, 213, 278, 284, 312, 386, 457, 564])

In [132]:
np.array(failed_clips2)

array([ 74,  82, 126, 149, 211, 213, 278, 284, 312, 386, 457, 564])

In [41]:
failed_clips3 = []
for i in tqdm(failed_clips2):
    num = npsave2(i)
    
    if num != None:
        failed_clips3.append(num)
    
    

100%|███████████████████████████████████████████| 13/13 [00:15<00:00,  1.23s/it]


In [44]:
failed_clips3

[74, 82, 126, 149, 211, 213, 278, 284, 312, 386, 457, 564]

In [46]:
set(failed_clips2)-set(failed_clips3)

{8}

In [47]:
failed_clips3 = []
for i in tqdm(failed_clips2):
    num = npsave2(i)
    
    if num != None:
        failed_clips3.append(num)

100%|███████████████████████████████████████████| 13/13 [00:15<00:00,  1.22s/it]


In [134]:
def gamma(img, num):
    g = num
    #g = float(input("감마 값 : "))
    out = img.copy()
    #out = img.astype(np.float)
    out = ((out / 255) ** (1 / g)) * 255
    out = out.astype(np.uint8)
    return out

In [174]:
def play_bright(cap, g):
    data = np.array([])

    while cap.isOpened():
        ret, img = cap.read()

        if ret:
            imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            imgRGB = gamma(imgRGB, g)
            #imgRGB = saturate_contrast(imgRGB, 1.1)
            results = hands.process(imgRGB)

            ###### Data 
            if results.multi_hand_landmarks:
                for res in results.multi_hand_landmarks:
                    mpDraw.draw_landmarks(img, res, mpHands.HAND_CONNECTIONS)
                    joint = np.zeros((21, 3))
                    for id, lm in enumerate(res.landmark):
                        joint[id] = [lm.x, lm.y, lm.z]

                    # Compute Angles (1) (2) (3) Fingers
                    v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19], :3] # Parent joint
                    v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], :3] # Child joint
                    v = v2 - v1

                    v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

                    angle = np.arccos(np.einsum('nt,nt->n',
                        v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                        v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) # (15,0)
                    
                    # rad 2 deg
                    angle = np.degrees(angle)
                    angle = np.array(angle, dtype=np.int32)

                    tmp = np.array([angle])
                    tmp = tmp.flatten()
                    data = np.append(data, tmp)
                    
        else:
            cap.release()
            break
        img2 = cv2.cvtColor(imgRGB, cv2.COLOR_BGR2RGB)
        cv2.imshow("Image", img2)
        cv2.waitKey(1)
        cv2.destroyAllWindows()
    return np.array(data, dtype=np.int32)

In [175]:
def npsave_bright(n, g):
    cp = fileload(n)
    tmp = play_bright(cp, g)
    if (len(tmp) > 0):
        if n<10:
            with open('/Users/user/PycharmProjects/DL_hands/open/data2/fingers_info00'+str(n), "wb") as f:
                np.save(f, tmp)
        elif n<100:
            with open('/Users/user/PycharmProjects/DL_hands/open/data2/fingers_info0'+str(n), "wb") as f:
                np.save(f, tmp)
        else:
            with open('/Users/user/PycharmProjects/DL_hands/open/data2/fingers_info'+str(n), "wb") as f:
                np.save(f, tmp)
        return None
    else:
        return n

In [180]:
while failed_clips2:
    for i in failed_clips2:
        for j in range(8, 20):
            num = npsave_bright(i, j/10)
            
            if num == None:
                failed_clips2.remove(i)
                break


KeyboardInterrupt: 

In [181]:
failed_clips2


[213, 278, 312, 457]